In [178]:
import requests
import os
import json
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from pprint import pprint
import pytz
import seaborn as sb
import xgboost as xgb

from sklearn.metrics import classification_report



from datetime import datetime
from datetime import timedelta
from datetime import date
from sklearn.preprocessing import LabelEncoder
import time
from PIL import Image
from IPython.display import display
from concurrent.futures import ThreadPoolExecutor, as_completed

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import json

pd.options.mode.chained_assignment = None
import re
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, RFE, mutual_info_classif
import requests



from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score


pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)


In [ ]:


thanksgiving_standings = pd.read_csv(
    "thanksgiving_standings.csv"
)
final_standings = pd.read_csv("final_standings_all.csv", engine="pyarrow")


all_games_summary = pd.read_csv("history_game_summary.csv", engine="pyarrow")

In [330]:
thanksgiving_standings.head()

,conferenceName,conferenceSequence,divisionName,divisionSequence,gamesPlayed,goalDifferential,goalDifferentialPctg,goalAgainst,goalFor,goalsForPctg,pointPctg,points,streakCode,streakCount,winPctg,teamName.default,tri_code,date,season_id
0,Eastern,1,Northeast,1,22,20,0.909091,43,63,2.863636,0.636364,28,W,3,0.545455,Toronto Maple Leafs,TOR,11/23/2000,20002001
1,Eastern,2,Atlantic,1,21,-2,-0.095238,58,56,2.666667,0.571429,24,W,6,0.476190,Philadelphia Flyers,PHI,11/23/2000,20002001
2,Eastern,3,Southeast,1,20,-11,-0.550000,58,47,2.350000,0.450000,18,W,1,0.350000,Carolina Hurricanes,CAR,11/23/2000,20002001
3,Eastern,4,Northeast,2,21,16,0.761905,52,68,3.238095,0.619048,26,L,1,0.523810,Ottawa Senators,OTT,11/23/2000,20002001
4,Eastern,5,Northeast,3,19,6,0.315789,45,51,2.684211,0.605263,23,L,2,0.526316,Buffalo Sabres,BUF,11/23/2000,20002001


In [331]:
final_standings = final_standings[['conferenceName', 'conferenceSequence', 'divisionName',
       'divisionSequence', 'gamesPlayed', 'goalDifferential',
       'goalDifferentialPctg', 'goalAgainst', 'goalFor', 'goalsForPctg',
       'pointPctg', 'points', 'streakCode', 'streakCount', 'winPctg',
       'teamName.default', 'teamAbbrev.default', 'end_date', 'season_id', 'clinchIndicator']]

train_game_summary = all_games_summary[['game_id', 'season', 'awayTeam.id',
       'awayTeam.abbrev','homeTeam.id', 'homeTeam.abbrev','awayTeam.score', 'homeTeam.score','game_date_time', 'start_time', 'game_date']]


In [332]:
thanksgiving_standings = thanksgiving_standings.rename(columns = {'conferenceName':'season_conference', 'conferenceSequence': 'tx_conference_rank', 'divisionName': 'season_division', 
                                                                  'divisionSequence': 'tx_division_rank', 'gamesPlayed':'tx_gp', 'goalDifferential':'tx_goal_diff', 
                                                                  'goalDifferentialPctg':'tx_goal_diff_pct', 'goalAgainst':'tx_ga', 'goalFor' : 'tx_gf', 'goalsForPctg': 'tx_gf_pctg',
                                                                  'pointPctg':'tx_points_pct', 'streakCode':'tx_streak_code', 'streakCount': 'tx_streak_count', 'winPctg':'tx_win_pctg',
																   'teamName.default':'team_name', 'date':'thanksgiving_date', 'points': 'tx_points'})

final_standings = final_standings.rename(columns = {'conferenceName':'season_conference', 'conferenceSequence': 'end_conference_rank', 'divisionName': 'season_division', 
                                                                  'divisionSequence': 'end_division_rank', 'gamesPlayed':'end_gp', 'goalDifferential':'end_goal_diff', 
                                                                  'goalDifferentialPctg':'end_goal_diff_pct', 'goalAgainst':'end_ga', 'goalFor' : 'end_gf', 'goalsForPctg': 'end_gf_pctg',
                                                                  'pointPctg':'end_points_pct', 'streakCode':'end_streak_code', 'streakCount': 'end_streak_count', 'winPctg':'end_win_pctg',
																   'teamName.default':'team_name', 'teamAbbrev.default':'tri_code', 'points':'end_points', 'clinchIndicator': 'made_playoffs'})
train_game_summary = train_game_summary.rename(columns = {'season': 'season_id'})


In [333]:
# final_standings['made_playoffs'] = np.where(final_standings['made_playoffs'] != "None", 1, 0).astype(int)

# Training Data Frame

In [334]:
thanksgiving_training = thanksgiving_standings[thanksgiving_standings['season_id'] != 20242025]
final_training = final_standings[final_standings['season_id'] != 20242025]

train_df = thanksgiving_training.merge(final_training, on = ['season_id','tri_code', 'team_name', 'season_conference', 'season_division'], how = 'left')

train_df["goal_diff_pct_change"] = (
    train_df["tx_goal_diff_pct"]
    - train_df["end_goal_diff_pct"]
)
train_df["season_id"] = (
    train_df["season_id"].fillna(0).astype("int64")
)
train_df['made_playoffs'] = np.where(pd.isna(train_df['made_playoffs']), 0, 1).astype(int)
train_df.head()

,season_conference,tx_conference_rank,season_division,tx_division_rank,tx_gp,tx_goal_diff,tx_goal_diff_pct,tx_ga,tx_gf,tx_gf_pctg,tx_points_pct,tx_points,tx_streak_code,tx_streak_count,tx_win_pctg,team_name,tri_code,thanksgiving_date,season_id,end_conference_rank,end_division_rank,end_gp,end_goal_diff,end_goal_diff_pct,end_ga,end_gf,end_gf_pctg,end_points_pct,end_points,end_streak_code,end_streak_count,end_win_pctg,end_date,made_playoffs,goal_diff_pct_change
0,Eastern,1,Northeast,1,22,20,0.909091,43,63,2.863636,0.636364,28,W,3,0.545455,Toronto Maple Leafs,TOR,11/23/2000,20002001,7.0,3.0,82.0,25.0,0.304878,207.0,232.0,2.829268,0.548780,90.0,L,1.0,0.451220,2001-04-08,1,0.604213
1,Eastern,2,Atlantic,1,21,-2,-0.095238,58,56,2.666667,0.571429,24,W,6,0.476190,Philadelphia Flyers,PHI,11/23/2000,20002001,4.0,2.0,82.0,33.0,0.402439,207.0,240.0,2.926829,0.609756,100.0,W,2.0,0.524390,2001-04-08,1,-0.497677
2,Eastern,3,Southeast,1,20,-11,-0.550000,58,47,2.350000,0.450000,18,W,1,0.350000,Carolina Hurricanes,CAR,11/23/2000,20002001,8.0,2.0,82.0,-13.0,-0.158537,225.0,212.0,2.585366,0.536585,88.0,L,1.0,0.463415,2001-04-08,1,-0.391463
3,Eastern,4,Northeast,2,21,16,0.761905,52,68,3.238095,0.619048,26,L,1,0.523810,Ottawa Senators,OTT,11/23/2000,20002001,2.0,1.0,82.0,69.0,0.841463,205.0,274.0,3.341463,0.664634,109.0,W,2.0,0.585366,2001-04-08,1,-0.079558
4,Eastern,5,Northeast,3,19,6,0.315789,45,51,2.684211,0.605263,23,L,2,0.526316,Buffalo Sabres,BUF,11/23/2000,20002001,5.0,2.0,82.0,34.0,0.414634,184.0,218.0,2.658537,0.597561,98.0,L,1.0,0.560976,2001-04-08,1,-0.098845


In [335]:
#Identify where the team is in relation to the playoff structure. status_a = in playoffs by thanksgiving, status_b = in playoffs at end.

train_df["tx_playoff_spot"] = np.where(
    train_df["tx_conference_rank"] < 9, 1, 0
)



In [336]:
# Filter teams in playoff position at Thanksgiving
playoff_at_thanksgiving = train_df[
    train_df["tx_playoff_spot"] == 1
]

# Calculate odds of making playoffs
probability = playoff_at_thanksgiving["made_playoffs"].mean()  # Mean gives the proportion of 1s

print(f"Probability of making playoffs if in playoff position at Thanksgiving: {probability:.2%}")


Probability of making playoffs if in playoff position at Thanksgiving: 70.11%


In [337]:
# is there a difference between the conferences?
conference_prob = playoff_at_thanksgiving.groupby("season_conference")["made_playoffs"].mean()
print(conference_prob)


season_conference
Eastern    0.711957
Western    0.690217
Name: made_playoffs, dtype: float64


In [338]:
#Identifying teams where their status changes: no change = 0 (either still in or still out), drop out = -1, late_surge = 1
train_df["status_change"] = np.where(
    (train_df["tx_playoff_spot"] == 0)
    & (train_df["made_playoffs"] == 1),  # Condition for position == 1
    1,  # Value if true
    np.where(
        (train_df["tx_playoff_spot"] == 1)
        & (train_df["made_playoffs"] == 0),  # Condition for position == -1
        -1,  # Value if true
        0,  # Default value if neither condition is true
    ),
)

train_df.head()



,season_conference,tx_conference_rank,season_division,tx_division_rank,tx_gp,tx_goal_diff,tx_goal_diff_pct,tx_ga,tx_gf,tx_gf_pctg,tx_points_pct,tx_points,tx_streak_code,tx_streak_count,tx_win_pctg,team_name,tri_code,thanksgiving_date,season_id,end_conference_rank,end_division_rank,end_gp,end_goal_diff,end_goal_diff_pct,end_ga,end_gf,end_gf_pctg,end_points_pct,end_points,end_streak_code,end_streak_count,end_win_pctg,end_date,made_playoffs,goal_diff_pct_change,tx_playoff_spot,status_change
0,Eastern,1,Northeast,1,22,20,0.909091,43,63,2.863636,0.636364,28,W,3,0.545455,Toronto Maple Leafs,TOR,11/23/2000,20002001,7.0,3.0,82.0,25.0,0.304878,207.0,232.0,2.829268,0.548780,90.0,L,1.0,0.451220,2001-04-08,1,0.604213,1,0
1,Eastern,2,Atlantic,1,21,-2,-0.095238,58,56,2.666667,0.571429,24,W,6,0.476190,Philadelphia Flyers,PHI,11/23/2000,20002001,4.0,2.0,82.0,33.0,0.402439,207.0,240.0,2.926829,0.609756,100.0,W,2.0,0.524390,2001-04-08,1,-0.497677,1,0
2,Eastern,3,Southeast,1,20,-11,-0.550000,58,47,2.350000,0.450000,18,W,1,0.350000,Carolina Hurricanes,CAR,11/23/2000,20002001,8.0,2.0,82.0,-13.0,-0.158537,225.0,212.0,2.585366,0.536585,88.0,L,1.0,0.463415,2001-04-08,1,-0.391463,1,0
3,Eastern,4,Northeast,2,21,16,0.761905,52,68,3.238095,0.619048,26,L,1,0.523810,Ottawa Senators,OTT,11/23/2000,20002001,2.0,1.0,82.0,69.0,0.841463,205.0,274.0,3.341463,0.664634,109.0,W,2.0,0.585366,2001-04-08,1,-0.079558,1,0
4,Eastern,5,Northeast,3,19,6,0.315789,45,51,2.684211,0.605263,23,L,2,0.526316,Buffalo Sabres,BUF,11/23/2000,20002001,5.0,2.0,82.0,34.0,0.414634,184.0,218.0,2.658537,0.597561,98.0,L,1.0,0.560976,2001-04-08,1,-0.098845,1,0


In [339]:
# remove the teams that are not in playoff position ever
playoff_contenders = train_df.query("tx_playoff_spot != 0 & made_playoffs !=0")
playoff_contenders.head()

,season_conference,tx_conference_rank,season_division,tx_division_rank,tx_gp,tx_goal_diff,tx_goal_diff_pct,tx_ga,tx_gf,tx_gf_pctg,tx_points_pct,tx_points,tx_streak_code,tx_streak_count,tx_win_pctg,team_name,tri_code,thanksgiving_date,season_id,end_conference_rank,end_division_rank,end_gp,end_goal_diff,end_goal_diff_pct,end_ga,end_gf,end_gf_pctg,end_points_pct,end_points,end_streak_code,end_streak_count,end_win_pctg,end_date,made_playoffs,goal_diff_pct_change,tx_playoff_spot,status_change
0,Eastern,1,Northeast,1,22,20,0.909091,43,63,2.863636,0.636364,28,W,3,0.545455,Toronto Maple Leafs,TOR,11/23/2000,20002001,7.0,3.0,82.0,25.0,0.304878,207.0,232.0,2.829268,0.548780,90.0,L,1.0,0.451220,2001-04-08,1,0.604213,1,0
1,Eastern,2,Atlantic,1,21,-2,-0.095238,58,56,2.666667,0.571429,24,W,6,0.476190,Philadelphia Flyers,PHI,11/23/2000,20002001,4.0,2.0,82.0,33.0,0.402439,207.0,240.0,2.926829,0.609756,100.0,W,2.0,0.524390,2001-04-08,1,-0.497677,1,0
2,Eastern,3,Southeast,1,20,-11,-0.550000,58,47,2.350000,0.450000,18,W,1,0.350000,Carolina Hurricanes,CAR,11/23/2000,20002001,8.0,2.0,82.0,-13.0,-0.158537,225.0,212.0,2.585366,0.536585,88.0,L,1.0,0.463415,2001-04-08,1,-0.391463,1,0
3,Eastern,4,Northeast,2,21,16,0.761905,52,68,3.238095,0.619048,26,L,1,0.523810,Ottawa Senators,OTT,11/23/2000,20002001,2.0,1.0,82.0,69.0,0.841463,205.0,274.0,3.341463,0.664634,109.0,W,2.0,0.585366,2001-04-08,1,-0.079558,1,0
4,Eastern,5,Northeast,3,19,6,0.315789,45,51,2.684211,0.605263,23,L,2,0.526316,Buffalo Sabres,BUF,11/23/2000,20002001,5.0,2.0,82.0,34.0,0.414634,184.0,218.0,2.658537,0.597561,98.0,L,1.0,0.560976,2001-04-08,1,-0.098845,1,0


In [340]:
#this is identifying the line
# Calculate 8th spot points for each season and conference during Thanksgiving
tx_8th_spot = (
    train_df[train_df['tx_conference_rank'] == 8]
    .groupby(["season_id", "season_conference"])["tx_points"]
    .first()
    .to_dict()
)

# Calculate 8th spot points for each season and conference at season end
end_8th_spot = (
    train_df[train_df['end_conference_rank'] == 8]
    .groupby(["season_id", "season_conference"])["end_points"]
    .first()
    .to_dict()
)

# Create a combined key for mapping
season_conference_keys = list(
    zip(train_df["season_id"], train_df['season_conference'])
)

# Map the 8th spot points using the combined keys
train_df['tx_8th_spot_points'] = [tx_8th_spot.get(key, None) for key in season_conference_keys]
train_df['end_8th_spot_points'] = [end_8th_spot.get(key, None) for key in season_conference_keys]


## Status Change Teams

In [341]:

# Filter for late surge teams by conference
late_surge_east = train_df[
    (train_df["status_change"] == 1)
    & (train_df["season_conference"] == "Eastern")
]

late_surge_west = train_df[
    (train_df["status_change"]  == 1)
    & (train_df["season_conference"] == "Western")
]
# Filter for drop out teams by conference
drop_out_east = train_df[
    (train_df["status_change"] == -1)
    & (train_df["season_conference"] == "Eastern")
]

drop_out_west = train_df[
    (train_df["status_change"] == -1)
    & (train_df["season_conference"] == "Western")
]

# Eastern
tx_avg_distanceE = round(
    (late_surge_east["tx_points"] - late_surge_east["tx_8th_spot_points"]).mean(), 2
)  # on average, how many points out of playoffs were the teams?

tx_med_distanceE = round(
    (late_surge_east["tx_points"] - late_surge_east["tx_8th_spot_points"]).median(), 2
)

tx_mode_distanceE = (
    late_surge_east["tx_points"] - late_surge_east["tx_8th_spot_points"]
).mode()[0]

tx_points_pct_changeE = round(
    (late_surge_east["tx_points_pct"] - late_surge_east["end_points_pct"]).mean(), 2
)
# Output summary for late-surge teams
print(f"Eastern Late Surge Teams:")
print(f"Average points distance to 8th spot: {tx_avg_distanceE}")
print(f"Median points distance to 8th spot: {tx_med_distanceE}")
print(f"Most frequent Points distance to 8th spot: {tx_mode_distanceE}")
print(f"Avg change in points percentage: {tx_points_pct_changeE}")

# Check the distance from 8th place at Thanksgiving
tx_distances = late_surge_west["tx_points"] - late_surge_west["tx_8th_spot_points"]

tx_avg_distanceW = round(tx_distances.mean(), 2) if not tx_distances.empty else None
tx_med_distanceW = round(tx_distances.median(), 2) if not tx_distances.empty else None

tx_mode_series = tx_distances.mode()
tx_mode_distanceW = tx_mode_series[0] if not tx_mode_series.empty else None

# Change in points percentage
if not late_surge_west.empty:
    tx_points_pct_changeW = round(
        (late_surge_west["tx_points_pct"] - late_surge_west["end_points_pct"]).mean(), 2
    )
else:
    tx_points_pct_changeW = None

# Output
print(f"Western Late Surge Teams:")
print(f"Average points distance to 8th spot: {tx_avg_distanceW}")
print(f"Median points distance to 8th spot: {tx_med_distanceW}")
print(f"Most frequent Points distance to 8th spot: {tx_mode_distanceW}")
print(f"Avg change in points percentage: {tx_points_pct_changeW}")


Eastern Late Surge Teams:
Average points distance to 8th spot: -2.89
Median points distance to 8th spot: -3.0
Most frequent Points distance to 8th spot: -2
Avg change in points percentage: -0.1
Western Late Surge Teams:
Average points distance to 8th spot: -2.05
Median points distance to 8th spot: -1.0
Most frequent Points distance to 8th spot: 0
Avg change in points percentage: -0.1


In [342]:
# East

tx_avg_distance_dropE = round(
    (drop_out_east["tx_points"] - drop_out_east["tx_8th_spot_points"]).mean(), 2
)  # on average, how many points out of playoffs were the teams?

tx_med_distance_dropE = round(
    (drop_out_east["tx_points"] - drop_out_east["tx_8th_spot_points"]).median(), 2
)

tx_mode_distance_dropE = (
    drop_out_east["tx_points"] - drop_out_east["tx_8th_spot_points"]
).mode()[0]

tx_points_pct_change_dropE = round(
    (drop_out_east["tx_points_pct"] - drop_out_east["end_points_pct"]).mean(), 2
)

# West
tx_avg_distance_dropW = round(
    (drop_out_west["tx_points"] - drop_out_west["tx_8th_spot_points"]).mean(), 2
)  # on average, how many points out of playoffs were the teams?

tx_med_distance_dropW = round(
    (drop_out_west["tx_points"] - drop_out_west["tx_8th_spot_points"]).median(), 2
)

tx_mode_distance_dropW = (
    drop_out_west["tx_points"] - drop_out_west["tx_8th_spot_points"]
).mode()[0]

tx_points_pct_change_dropW = round(
    (drop_out_west["tx_points_pct"] - drop_out_west["end_points_pct"]).mean(), 2
)

# Output summary for late-surge teams
print(f"Drop Out Teams East:")
print(f"Average points distance to 8th spot: {tx_avg_distance_dropE}")
print(f"Median points distance to 8th spot: {tx_med_distance_dropE}")
print(f"Most frequent Points distance to 8th spot: {tx_mode_distance_dropE}")
print(f"Avg change in points percentage: {tx_points_pct_change_dropE}")

# Output summary for late-surge teams
print(f"Drop Out Teams West:")
print(f"Average points distance to 8th spot: {tx_avg_distance_dropW}")
print(f"Median points distance to 8th spot: {tx_med_distance_dropW}")
print(f"Most frequent Points distance to 8th spot: {tx_mode_distance_dropW}")
print(f"Avg change in points percentage: {tx_points_pct_change_dropW}")

Drop Out Teams East:
Average points distance to 8th spot: 2.43
Median points distance to 8th spot: 2.0
Most frequent Points distance to 8th spot: 0
Avg change in points percentage: 0.1
Drop Out Teams West:
Average points distance to 8th spot: 2.58
Median points distance to 8th spot: 2.0
Most frequent Points distance to 8th spot: 0
Avg change in points percentage: 0.09


In [343]:
drop_out_east["tx_points_distance"] = (
    drop_out_east["tx_points"] - drop_out_east["tx_8th_spot_points"]
)
drop_out_west["tx_points_distance"] = (
    drop_out_west["tx_points"] - drop_out_west["tx_8th_spot_points"]
)
late_surge_east["tx_points_distance"] = (
    late_surge_east["tx_points"] - late_surge_east["tx_8th_spot_points"]
)
late_surge_west["tx_points_distance"] = (
    late_surge_west["tx_points"] - late_surge_west["tx_8th_spot_points"]
)
# Find the minimum value of tx_points_distance
min_distance = late_surge_east["tx_points_distance"].min()

# Filter rows with the minimum tx_points_distance
teams_with_min_distance = late_surge_east[
    late_surge_east["tx_points_distance"] == min_distance
]

# Display team names and seasons
print(teams_with_min_distance[["team_name", "season_id", "tx_points_distance"]])
# Find the minimum value of tx_points_distance
min_distance = late_surge_west["tx_points_distance"].min()

# Filter rows with the minimum tx_points_distance
teams_with_min_distance = late_surge_west[
    late_surge_west["tx_points_distance"] == min_distance
]

# Display team names and seasons
print(teams_with_min_distance[["team_name", "season_id", "tx_points_distance"]])


               team_name  season_id  tx_points_distance
194  Washington Capitals   20072008                  -9
           team_name  season_id  tx_points_distance
697  Edmonton Oilers   20232024                 -10


In [344]:
# Create streak features
def calculate_streak(row, streak_code_col):
    """
    Calculates the streak value based on the streak code column provided.
    W = 1, L = -1, and other cases will be 0 (or missing streak).
    """
    if row[streak_code_col] == "W":
        return 1
    elif row[streak_code_col] == "L":
        return -1
    else:
        return 0

# Apply the function to calculate Thanksgiving streak
train_df["thanksgiving_streak"] = train_df.apply(
    lambda row: calculate_streak(row, "tx_streak_code"), axis=1
)

# Apply the function to calculate Season-end streak
train_df["end_streak"] = train_df.apply(
    lambda row: calculate_streak(row, "end_streak_code"), axis=1
)

# Drop the original streak code columns (optional)
train_df = train_df.drop(columns=["tx_streak_code", "end_streak_code"])
train_df["end_streak"] =train_df["end_streak"].astype(str)
train_df["thanksgiving_streak"] = train_df["thanksgiving_streak"].astype(str)
# Check the result
train_df.head()


,season_conference,tx_conference_rank,season_division,tx_division_rank,tx_gp,tx_goal_diff,tx_goal_diff_pct,tx_ga,tx_gf,tx_gf_pctg,tx_points_pct,tx_points,tx_streak_count,tx_win_pctg,team_name,tri_code,thanksgiving_date,season_id,end_conference_rank,end_division_rank,end_gp,end_goal_diff,end_goal_diff_pct,end_ga,end_gf,end_gf_pctg,end_points_pct,end_points,end_streak_count,end_win_pctg,end_date,made_playoffs,goal_diff_pct_change,tx_playoff_spot,status_change,tx_8th_spot_points,end_8th_spot_points,thanksgiving_streak,end_streak
0,Eastern,1,Northeast,1,22,20,0.909091,43,63,2.863636,0.636364,28,3,0.545455,Toronto Maple Leafs,TOR,11/23/2000,20002001,7.0,3.0,82.0,25.0,0.304878,207.0,232.0,2.829268,0.548780,90.0,1.0,0.451220,2001-04-08,1,0.604213,1,0,22,88.0,1,-1
1,Eastern,2,Atlantic,1,21,-2,-0.095238,58,56,2.666667,0.571429,24,6,0.476190,Philadelphia Flyers,PHI,11/23/2000,20002001,4.0,2.0,82.0,33.0,0.402439,207.0,240.0,2.926829,0.609756,100.0,2.0,0.524390,2001-04-08,1,-0.497677,1,0,22,88.0,1,1
2,Eastern,3,Southeast,1,20,-11,-0.550000,58,47,2.350000,0.450000,18,1,0.350000,Carolina Hurricanes,CAR,11/23/2000,20002001,8.0,2.0,82.0,-13.0,-0.158537,225.0,212.0,2.585366,0.536585,88.0,1.0,0.463415,2001-04-08,1,-0.391463,1,0,22,88.0,1,-1
3,Eastern,4,Northeast,2,21,16,0.761905,52,68,3.238095,0.619048,26,1,0.523810,Ottawa Senators,OTT,11/23/2000,20002001,2.0,1.0,82.0,69.0,0.841463,205.0,274.0,3.341463,0.664634,109.0,2.0,0.585366,2001-04-08,1,-0.079558,1,0,22,88.0,-1,1
4,Eastern,5,Northeast,3,19,6,0.315789,45,51,2.684211,0.605263,23,2,0.526316,Buffalo Sabres,BUF,11/23/2000,20002001,5.0,2.0,82.0,34.0,0.414634,184.0,218.0,2.658537,0.597561,98.0,1.0,0.560976,2001-04-08,1,-0.098845,1,0,22,88.0,-1,-1


# Strength of Schedule

In [345]:
# Create dictionaries keyed by team for mapping
tx_win_pctg_map = train_df.set_index("tri_code")["tx_win_pctg"].to_dict()
end_win_pctg_map = train_df.set_index("tri_code")["end_win_pctg"].to_dict()

# Map win % to each opponent in game log
train_game_summary["away_tx_win_pctg"] = train_game_summary["awayTeam.abbrev"].map(tx_win_pctg_map)
train_game_summary["home_tx_win_pctg"] = train_game_summary["homeTeam.abbrev"].map(tx_win_pctg_map)

train_game_summary["away_end_win_pctg"] = train_game_summary["awayTeam.abbrev"].map(end_win_pctg_map)
train_game_summary["home_end_win_pctg"] = train_game_summary["homeTeam.abbrev"].map(end_win_pctg_map)
train_game_summary.head()


,game_id,season_id,awayTeam.id,awayTeam.abbrev,homeTeam.id,homeTeam.abbrev,awayTeam.score,homeTeam.score,game_date_time,start_time,game_date,away_tx_win_pctg,home_tx_win_pctg,away_end_win_pctg,home_end_win_pctg
0,2000020001,20002001,21,COL,25,DAL,2,2,2000-10-04 23:00:00+00:00,7:00 pm,2000-10-04,0.666667,0.666667,0.609756,0.634146
1,2000020002,20002001,9,OTT,6,BOS,4,4,2000-10-05 23:00:00+00:00,7:00 pm,2000-10-05,0.533333,0.777778,0.451220,0.573171
2,2000020003,20002001,16,CHI,7,BUF,2,4,2000-10-05 23:00:00+00:00,7:00 pm,2000-10-05,0.294118,0.421053,0.280488,0.475610
3,2000020004,20002001,23,VAN,4,PHI,3,6,2000-10-05 23:00:00+00:00,7:00 pm,2000-10-05,0.650000,0.526316,0.609756,0.463415
4,2000020005,20002001,17,DET,20,CGY,4,3,2000-10-05 23:00:00+00:00,7:00 pm,2000-10-05,0.500000,0.368421,0.500000,0.463415


In [346]:
# tx_SOS = average win % of opponents up to Thanksgiving

away_tx_sos = (
    train_game_summary.groupby(["season_id", "awayTeam.abbrev"])["home_tx_win_pctg"]
    .mean()
    .reset_index()
    .rename(columns={"awayTeam.abbrev": "tri_code", "home_tx_win_pctg": "tx_SOS"})
)

home_tx_sos = (
    train_game_summary.groupby(["season_id", "homeTeam.abbrev"])["away_tx_win_pctg"]
    .mean()
    .reset_index()
    .rename(columns={"homeTeam.abbrev": "tri_code", "away_tx_win_pctg": "tx_SOS"})
)

tx_sos = pd.concat([away_tx_sos, home_tx_sos])
tx_sos = tx_sos.groupby(["season_id", "tri_code"])["tx_SOS"].mean().reset_index()
away_end_sos = (
    train_game_summary.groupby(["season_id", "awayTeam.abbrev"])["home_end_win_pctg"]
    .mean()
    .reset_index()
    .rename(columns={"awayTeam.abbrev": "tri_code", "home_end_win_pctg": "end_SOS"})
)

home_end_sos = (
    train_game_summary.groupby(["season_id", "homeTeam.abbrev"])["away_end_win_pctg"]
    .mean()
    .reset_index()
    .rename(columns={"homeTeam.abbrev": "tri_code", "away_end_win_pctg": "end_SOS"})
)

end_sos = pd.concat([away_end_sos, home_end_sos])
end_sos = end_sos.groupby(["season_id", "tri_code"])["end_SOS"].mean().reset_index()
train_df = train_df.merge(tx_sos, on=["season_id", "tri_code"], how="left")
train_df = train_df.merge(end_sos, on=["season_id", "tri_code"], how="left")


# 2. Merge Thanksgiving date into the game log
thanksgiving_lookup = train_df[["season_id", "thanksgiving_date"]].drop_duplicates()
train_game_summary = train_game_summary.merge(thanksgiving_lookup, on="season_id", how="left")
# Convert game_date and thanksgiving_date to datetime.date
train_game_summary["game_date"] = pd.to_datetime(train_game_summary["game_date"]).dt.date
train_game_summary["thanksgiving_date"] = pd.to_datetime(train_game_summary["thanksgiving_date"]).dt.date


post_tx_games = train_game_summary[train_game_summary["game_date"] > train_game_summary["thanksgiving_date"]]


# 4. Map end-of-season win % to each opponent
post_tx_games["away_end_win_pctg"] = post_tx_games["awayTeam.abbrev"].map(end_win_pctg_map)
post_tx_games["home_end_win_pctg"] = post_tx_games["homeTeam.abbrev"].map(end_win_pctg_map)

# 5. Compute post-TX SOS for each team
away_post_tx_sos = (
    post_tx_games.groupby(["season_id", "awayTeam.abbrev"])["home_end_win_pctg"]
    .mean()
    .reset_index()
    .rename(columns={"awayTeam.abbrev": "tri_code", "home_end_win_pctg": "post_tx_SOS"})
)

home_post_tx_sos = (
    post_tx_games.groupby(["season_id", "homeTeam.abbrev"])["away_end_win_pctg"]
    .mean()
    .reset_index()
    .rename(columns={"homeTeam.abbrev": "tri_code", "away_end_win_pctg": "post_tx_SOS"})
)

# 6. Combine and average home/away SOS
combined_post_tx_sos = pd.concat([away_post_tx_sos, home_post_tx_sos])
post_tx_sos = combined_post_tx_sos.groupby(["season_id", "tri_code"])["post_tx_SOS"].mean().reset_index()

# 7. Merge back into train_df
train_df = train_df.merge(post_tx_sos, on=["season_id", "tri_code"], how="left")



train_df.head()


,season_conference,tx_conference_rank,season_division,tx_division_rank,tx_gp,tx_goal_diff,tx_goal_diff_pct,tx_ga,tx_gf,tx_gf_pctg,tx_points_pct,tx_points,tx_streak_count,tx_win_pctg,team_name,tri_code,thanksgiving_date,season_id,end_conference_rank,end_division_rank,end_gp,end_goal_diff,end_goal_diff_pct,end_ga,end_gf,end_gf_pctg,end_points_pct,end_points,end_streak_count,end_win_pctg,end_date,made_playoffs,goal_diff_pct_change,tx_playoff_spot,status_change,tx_8th_spot_points,end_8th_spot_points,thanksgiving_streak,end_streak,tx_SOS,end_SOS,post_tx_SOS
0,Eastern,1,Northeast,1,22,20,0.909091,43,63,2.863636,0.636364,28,3,0.545455,Toronto Maple Leafs,TOR,11/23/2000,20002001,7.0,3.0,82.0,25.0,0.304878,207.0,232.0,2.829268,0.548780,90.0,1.0,0.451220,2001-04-08,1,0.604213,1,0,22,88.0,1,-1,0.511389,0.499851,0.494438
1,Eastern,2,Atlantic,1,21,-2,-0.095238,58,56,2.666667,0.571429,24,6,0.476190,Philadelphia Flyers,PHI,11/23/2000,20002001,4.0,2.0,82.0,33.0,0.402439,207.0,240.0,2.926829,0.609756,100.0,2.0,0.524390,2001-04-08,1,-0.497677,1,0,22,88.0,1,1,0.519054,0.508328,0.508286
2,Eastern,3,Southeast,1,20,-11,-0.550000,58,47,2.350000,0.450000,18,1,0.350000,Carolina Hurricanes,CAR,11/23/2000,20002001,8.0,2.0,82.0,-13.0,-0.158537,225.0,212.0,2.585366,0.536585,88.0,1.0,0.463415,2001-04-08,1,-0.391463,1,0,22,88.0,1,-1,0.513061,0.489292,0.488529
3,Eastern,4,Northeast,2,21,16,0.761905,52,68,3.238095,0.619048,26,1,0.523810,Ottawa Senators,OTT,11/23/2000,20002001,2.0,1.0,82.0,69.0,0.841463,205.0,274.0,3.341463,0.664634,109.0,2.0,0.585366,2001-04-08,1,-0.079558,1,0,22,88.0,-1,1,0.515156,0.502380,0.489883
4,Eastern,5,Northeast,3,19,6,0.315789,45,51,2.684211,0.605263,23,2,0.526316,Buffalo Sabres,BUF,11/23/2000,20002001,5.0,2.0,82.0,34.0,0.414634,184.0,218.0,2.658537,0.597561,98.0,1.0,0.560976,2001-04-08,1,-0.098845,1,0,22,88.0,-1,-1,0.528129,0.501785,0.509830


In [359]:
# num_teams_per_season_train = (
#     train_df.groupby("season_id")["team_name"].nunique().reset_index()
# )
# num_teams_per_season_train.rename(columns={"team_name": "num_teams"}, inplace=True)

# train_df = train_df.merge(
#     num_teams_per_season_train, on="season_id", how="left"
# )

conference_sizes = (
    train_df.groupby(["season_id", "season_conference"])["tri_code"]
    .nunique()
    .reset_index()
    .rename(columns={"tri_code": "conference_team_count"})
)
train_df = train_df.merge(conference_sizes, on=["season_id", "season_conference"], how="left")

train_df["conf_playoff_chance_baseline"] = 8 / train_df["conference_team_count"]


train_df.head()

,season_conference,tx_conference_rank,season_division,tx_division_rank,tx_gp,tx_goal_diff,tx_goal_diff_pct,tx_ga,tx_gf,tx_gf_pctg,tx_points_pct,tx_points,tx_streak_count,tx_win_pctg,team_name,tri_code,thanksgiving_date,season_id,end_conference_rank,end_division_rank,end_gp,end_goal_diff,end_goal_diff_pct,end_ga,end_gf,end_gf_pctg,end_points_pct,end_points,end_streak_count,end_win_pctg,end_date,made_playoffs,goal_diff_pct_change,tx_playoff_spot,status_change,tx_8th_spot_points,end_8th_spot_points,thanksgiving_streak,end_streak,tx_SOS,end_SOS,post_tx_SOS,num_teams,conference_team_count,conf_playoff_chance_baseline
0,Eastern,1,Northeast,1,22,20,0.909091,43,63,2.863636,0.636364,28,3,0.545455,Toronto Maple Leafs,TOR,11/23/2000,20002001,7.0,3.0,82.0,25.0,0.304878,207.0,232.0,2.829268,0.548780,90.0,1.0,0.451220,2001-04-08,1,0.604213,1,0,22,88.0,1,-1,0.511389,0.499851,0.494438,30,15,0.533333
1,Eastern,2,Atlantic,1,21,-2,-0.095238,58,56,2.666667,0.571429,24,6,0.476190,Philadelphia Flyers,PHI,11/23/2000,20002001,4.0,2.0,82.0,33.0,0.402439,207.0,240.0,2.926829,0.609756,100.0,2.0,0.524390,2001-04-08,1,-0.497677,1,0,22,88.0,1,1,0.519054,0.508328,0.508286,30,15,0.533333
2,Eastern,3,Southeast,1,20,-11,-0.550000,58,47,2.350000,0.450000,18,1,0.350000,Carolina Hurricanes,CAR,11/23/2000,20002001,8.0,2.0,82.0,-13.0,-0.158537,225.0,212.0,2.585366,0.536585,88.0,1.0,0.463415,2001-04-08,1,-0.391463,1,0,22,88.0,1,-1,0.513061,0.489292,0.488529,30,15,0.533333
3,Eastern,4,Northeast,2,21,16,0.761905,52,68,3.238095,0.619048,26,1,0.523810,Ottawa Senators,OTT,11/23/2000,20002001,2.0,1.0,82.0,69.0,0.841463,205.0,274.0,3.341463,0.664634,109.0,2.0,0.585366,2001-04-08,1,-0.079558,1,0,22,88.0,-1,1,0.515156,0.502380,0.489883,30,15,0.533333
4,Eastern,5,Northeast,3,19,6,0.315789,45,51,2.684211,0.605263,23,2,0.526316,Buffalo Sabres,BUF,11/23/2000,20002001,5.0,2.0,82.0,34.0,0.414634,184.0,218.0,2.658537,0.597561,98.0,1.0,0.560976,2001-04-08,1,-0.098845,1,0,22,88.0,-1,-1,0.528129,0.501785,0.509830,30,15,0.533333


### Random Forest

In [360]:
feature_cols = ['tx_points', 'tx_points_pct', 'tx_goal_diff', 
                'tx_conference_rank', 'thanksgiving_streak', 'end_streak', 'tx_SOS', 'end_SOS', 'post_tx_SOS', 'conf_playoff_chance_baseline']

#get the features
X = train_df[feature_cols]

#create the dummies
streak_dummies = pd.get_dummies(X[["thanksgiving_streak", "end_streak"]], prefix=["tx_", "end_"])
X = X.drop(columns=["thanksgiving_streak", "end_streak"])
X = pd.concat([X, streak_dummies], axis=1)
X = pd.get_dummies(X, drop_first=True)

#set the target
y = train_df['made_playoffs']

#train and test split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

rf = RandomForestClassifier(random_state=42, n_estimators=200)
rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)

print(classification_report(y_test, y_pred))
importances = rf.feature_importances_
for feature, importance in zip(X.columns, importances):
    print(f"{feature}: {importance:.4f}")


              precision    recall  f1-score   support

           0       0.69      0.71      0.70        69
           1       0.71      0.69      0.70        71

    accuracy                           0.70       140
   macro avg       0.70      0.70      0.70       140
weighted avg       0.70      0.70      0.70       140

tx_points: 0.1224
tx_points_pct: 0.2047
tx_goal_diff: 0.1592
tx_conference_rank: 0.1040
tx_SOS: 0.0955
end_SOS: 0.0932
post_tx_SOS: 0.0974
conf_playoff_chance_baseline: 0.0252
tx__-1: 0.0121
tx__0: 0.0074
tx__1: 0.0142
end__-1: 0.0131
end__0: 0.0317
end__1: 0.0199


In [361]:
#Random Forest parameter grid
rf_param_grid = {
	'n_estimators':[100,200],
	'max_depth':[None, 10, 20],
	'min_samples_split':[2,5],
	'min_samples_leaf':[1,2],
	'bootstrap':[True, False]
}
rf_model = RandomForestClassifier(random_state=42)

rf_grid_search = GridSearchCV(
	estimator=rf_model,
	param_grid=rf_param_grid,
	cv=5,
	n_jobs=-1,
	verbose=1
)

rf_grid_search.fit(X_train, y_train)

print("Random Forest Best Params:", rf_grid_search.best_params_)
print("Random Forest CV Score:", rf_grid_search.best_score_)

#Evaluate on test set
rf_best = rf_grid_search.best_estimator_
rf_preds = rf_best.predict(X_test)
print("Random Forest Test Report:\n", classification_report(y_test, rf_preds))


importances = rf.feature_importances_
for feature, importance in zip(X.columns, importances):
    print(f"{feature}: {importance:.4f}")


Fitting 5 folds for each of 48 candidates, totalling 240 fits
Random Forest Best Params: {'bootstrap': True, 'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 200}
Random Forest CV Score: 0.7767857142857144
Random Forest Test Report:
               precision    recall  f1-score   support

           0       0.72      0.74      0.73        69
           1       0.74      0.72      0.73        71

    accuracy                           0.73       140
   macro avg       0.73      0.73      0.73       140
weighted avg       0.73      0.73      0.73       140

tx_points: 0.1224
tx_points_pct: 0.2047
tx_goal_diff: 0.1592
tx_conference_rank: 0.1040
tx_SOS: 0.0955
end_SOS: 0.0932
post_tx_SOS: 0.0974
conf_playoff_chance_baseline: 0.0252
tx__-1: 0.0121
tx__0: 0.0074
tx__1: 0.0142
end__-1: 0.0131
end__0: 0.0317
end__1: 0.0199


In [371]:
# Initialize the XGBoost model
xgb_model = xgb.XGBClassifier(random_state=42)

# Fit the model
xgb_model.fit(X_train, y_train)

# Make predictions
y_pred = xgb_model.predict(X_test)

# Evaluate the model
print(classification_report(y_test, y_pred))

# Feature importance
importances = xgb_model.feature_importances_
for feature, importance in zip(X.columns, importances):
    print(f"{feature}: {importance:.4f}")


              precision    recall  f1-score   support

           0       0.70      0.70      0.70        69
           1       0.70      0.70      0.70        71

    accuracy                           0.70       140
   macro avg       0.70      0.70      0.70       140
weighted avg       0.70      0.70      0.70       140

tx_points: 0.0589
tx_points_pct: 0.1648
tx_goal_diff: 0.0681
tx_conference_rank: 0.0411
tx_SOS: 0.0419
end_SOS: 0.0485
post_tx_SOS: 0.0446
conf_playoff_chance_baseline: 0.0539
tx__-1: 0.0223
tx__0: 0.0245
tx__1: 0.0519
end__-1: 0.0251
end__0: 0.2973
end__1: 0.0571


In [ ]:
#XGBoost parameter grid
xbg_param_grid = {
    "max_depth": [3, 5, 7], # Depth of the tree
    "learning_rate": [0.01, 0.1, 0.2], # Step size shrinkage
    "n_estimators": [100, 200, 300], # Number of boosting rounds
    "subsample": [0.8, 1.0], # Subsample ratio of the training instances
    "colsample_bytree": [0.8, 1.0], # Subsample ratio of columns for each tree
    "gamma": [0, 1], # Minimum loss reduction
    "reg_alpha": [0, 0.01, 0.1], # L1 regularization term
    "reg_lambda": [1, 10], # L2 regularization term
}

xgb_model = xgb.XGBClassifier(
	random_state=42, use_label_encoder=False, eval_metric="logloss"
)

xgb_grid_search = GridSearchCV(
	estimator=xgb_model, 
	param_grid=xbg_param_grid,
	scoring='accuracy',
	cv=5,
	n_jobs=-1,
	verbose=1
)

xgb_grid_search.fit(X_train, y_train)

#Best XGBoost model
print("XGBoost Best Params:", xgb_grid_search.best_params_)
print("XGBoost CV Score:", xgb_grid_search.best_score_)

# Evaluate on test set
xgb_best = xgb_grid_search.best_estimator_
xgb_preds = xgb_best.predict(X_test)
print("XGBoost Test Report:\n", classification_report(y_test, xgb_preds))

Fitting 5 folds for each of 1296 candidates, totalling 6480 fits


# Current Season - Test Data

In [ ]:
test_df = thanksgiving_standings[thanksgiving_standings['season_id']==20242025]
test_game_summary = pd.read_csv("daily_game_summary.csv", engine="pyarrow")
test_df.columns

Index(['season_conference', 'tx_conference_rank', 'season_division',
       'tx_division_rank', 'tx_gp', 'tx_goal_diff', 'tx_goal_diff_pct',
       'tx_ga', 'tx_gf', 'tx_gf_pctg', 'tx_points_pct', 'tx_points',
       'tx_streak_code', 'tx_streak_count', 'tx_win_pctg', 'team_name',
       'tri_code', 'thanksgiving_date', 'season_id'],
      dtype='object')

In [ ]:
test_df = test_df.rename(columns = {'season_conference':'conference', 'tx_conference_rank': 'conference_rank', 'season_division': 'division', 
                                                                  'tx_division_rank': 'division_rank', 'tx_gp':'gp', 'tx_goal_diff':'goal_diff', 
                                                                  'tx_goal_diff_pct':'goal_diff_pct', 'tx_ga':'ga', 'tx_gf' : 'gf', 'tx_gf_pctg': 'gf_pctg',
                                                                  'tx_points_pct':'points_pct', 'tx_streak_code':'streak_code', 'tx_streak_count': 'streak_count', 'tx_win_pctg':'win_pctg',
																   'tx_points': 'points'})
test_df.head()

,conference,conference_rank,division,division_rank,gp,goal_diff,goal_diff_pct,ga,gf,gf_pctg,points_pct,points,streak_code,streak_count,win_pctg,team_name,tri_code,thanksgiving_date,season_id
700,Eastern,1,Metropolitan,1,22,30,1.363636,58,88,4.000000,0.750000,33,W,2,0.727273,Carolina Hurricanes,CAR,11/28/2024,20242025
701,Eastern,2,Metropolitan,2,25,19,0.760000,65,84,3.360000,0.640000,32,L,1,0.600000,New Jersey Devils,NJD,11/28/2024,20242025
702,Eastern,3,Metropolitan,3,22,30,1.363636,60,90,4.090909,0.704545,31,W,2,0.681818,Washington Capitals,WSH,11/28/2024,20242025
703,Eastern,4,Atlantic,1,22,8,0.363636,57,65,2.954545,0.636364,28,L,1,0.590909,Toronto Maple Leafs,TOR,11/28/2024,20242025
704,Eastern,5,Atlantic,2,23,3,0.130435,77,80,3.478261,0.586957,27,W,1,0.565217,Florida Panthers,FLA,11/28/2024,20242025


In [ ]:

test_df["playoff_spot"] = np.where(
    test_df["conference_rank"] < 9, 1, 0
)
test_df.head()

,conference,conference_rank,division,division_rank,gp,goal_diff,goal_diff_pct,ga,gf,gf_pctg,points_pct,points,streak_code,streak_count,win_pctg,team_name,tri_code,thanksgiving_date,season_id,playoff_spot
700,Eastern,1,Metropolitan,1,22,30,1.363636,58,88,4.000000,0.750000,33,W,2,0.727273,Carolina Hurricanes,CAR,11/28/2024,20242025,1
701,Eastern,2,Metropolitan,2,25,19,0.760000,65,84,3.360000,0.640000,32,L,1,0.600000,New Jersey Devils,NJD,11/28/2024,20242025,1
702,Eastern,3,Metropolitan,3,22,30,1.363636,60,90,4.090909,0.704545,31,W,2,0.681818,Washington Capitals,WSH,11/28/2024,20242025,1
703,Eastern,4,Atlantic,1,22,8,0.363636,57,65,2.954545,0.636364,28,L,1,0.590909,Toronto Maple Leafs,TOR,11/28/2024,20242025,1
704,Eastern,5,Atlantic,2,23,3,0.130435,77,80,3.478261,0.586957,27,W,1,0.565217,Florida Panthers,FLA,11/28/2024,20242025,1


In [ ]:
#Cutoff at thanksgiving
test_game_summary['game_date'] = pd.to_datetime(test_game_summary['game_date'])

# Filter to games on or before Nov 28, 2024
cutoff_date = pd.to_datetime("2024-11-28")
test_game_summary = test_game_summary[
    test_game_summary['game_date'] <= cutoff_date
]
test_game_summary=test_game_summary[['game_id', 'season', 'awayTeam.id',
       'awayTeam.abbrev','homeTeam.id', 'homeTeam.abbrev','awayTeam.score', 'homeTeam.score','game_date_time', 'start_time', 'game_date']]
test_game_summary = test_game_summary.rename(columns={'season':'season_id'})
test_game_summary.tail()

,game_id,season_id,awayTeam.id,awayTeam.abbrev,homeTeam.id,homeTeam.abbrev,awayTeam.score,homeTeam.score,game_date_time,start_time,game_date
356,2024020357,20242025,25,DAL,16,CHI,2,6,2024-11-27 20:30:00-05:00,8:30 PM,2024-11-27
357,2024020358,20242025,54,VGK,21,COL,1,2,2024-11-27 22:00:00-05:00,10:00 PM,2024-11-27
358,2024020359,20242025,52,WPG,26,LAK,1,4,2024-11-27 22:00:00-05:00,10:00 PM,2024-11-27
359,2024020360,20242025,24,ANA,55,SEA,5,2,2024-11-27 22:00:00-05:00,10:00 PM,2024-11-27
360,2024020361,20242025,9,OTT,28,SJS,4,3,2024-11-27 22:30:00-05:00,10:30 PM,2024-11-27


In [ ]:
# Create dictionaries keyed by team for mapping
win_pctg_map = test_df.set_index("tri_code")["win_pctg"].to_dict()

# Map win % to each opponent in game log
test_game_summary["away_win_pctg"] = test_game_summary["awayTeam.abbrev"].map(win_pctg_map)
test_game_summary["home_win_pctg"] = train_game_summary["homeTeam.abbrev"].map(win_pctg_map)


In [ ]:
# tx_SOS = average win % of opponents up to Thanksgiving

away_sos = (
    test_game_summary.groupby(["season_id", "awayTeam.abbrev"])["home_win_pctg"]
    .mean()
    .reset_index()
    .rename(columns={"awayTeam.abbrev": "tri_code", "home_win_pctg": "SOS"})
)

home_sos = (
    test_game_summary.groupby(["season_id", "homeTeam.abbrev"])["away_win_pctg"]
    .mean()
    .reset_index()
    .rename(columns={"homeTeam.abbrev": "tri_code", "away_win_pctg": "SOS"})
)

sos = pd.concat([away_sos, home_sos])
sos = sos.groupby(["season_id", "tri_code"])["SOS"].mean().reset_index()

test_df = test_df.merge(sos, on=["season_id", "tri_code"], how="left")
test_df.head()

,conference,conference_rank,division,division_rank,gp,goal_diff,goal_diff_pct,ga,gf,gf_pctg,points_pct,points,streak_code,streak_count,win_pctg,team_name,tri_code,thanksgiving_date,season_id,playoff_spot,SOS
0,Eastern,1,Metropolitan,1,22,30,1.363636,58,88,4.000000,0.750000,33,W,2,0.727273,Carolina Hurricanes,CAR,11/28/2024,20242025,1,0.494506
1,Eastern,2,Metropolitan,2,25,19,0.760000,65,84,3.360000,0.640000,32,L,1,0.600000,New Jersey Devils,NJD,11/28/2024,20242025,1,0.507510
2,Eastern,3,Metropolitan,3,22,30,1.363636,60,90,4.090909,0.704545,31,W,2,0.681818,Washington Capitals,WSH,11/28/2024,20242025,1,0.479726
3,Eastern,4,Atlantic,1,22,8,0.363636,57,65,2.954545,0.636364,28,L,1,0.590909,Toronto Maple Leafs,TOR,11/28/2024,20242025,1,0.507319
4,Eastern,5,Atlantic,2,23,3,0.130435,77,80,3.478261,0.586957,27,W,1,0.565217,Florida Panthers,FLA,11/28/2024,20242025,1,0.499246


In [ ]:

conference_sizes = (
    test_df.groupby(["season_id", "season_conference"])["tri_code"]
    .nunique()
    .reset_index()
    .rename(columns={"tri_code": "conference_team_count"})
)
test_df = test_df.merge(conference_sizes, on=["season_id", "season_conference"], how="left")

test_df["conf_playoff_chance_baseline"] = 8 / test_df["conference_team_count"]

test_df.head()

KeyError: 'season_conference'

# Predict

In [ ]:

X_test_final = test_df[feature_cols]

# One-hot encode streaks (like you did with train_df)
streak_dummies_test = pd.get_dummies(X_test_final[["thanksgiving_streak"]], prefix=["tx_"])
X_test_final = X_test_final.drop(columns=["thanksgiving_streak"])
X_test_final = pd.concat([X_test_final, streak_dummies_test], axis=1)

# One-hot encode any remaining categorical columns (drop_first=True)
X_test_final = pd.get_dummies(X_test_final, drop_first=True)

# Align test features with training features
X_test_final = X_test_final.reindex(columns=X.columns, fill_value=0)


In [ ]:
#Random Forest
rf_test_preds = rf_best.predict(X_test_final)
print("Random Forest Final Test Predictions:", rf_test_preds)

In [ ]:
#XGBoost
xgb_test_preds = xgb_best.predict(X_test_final)
print("XGBoost Final Test Predictions:", xgb_test_preds)

## Projected Points

In [ ]:
test_df["projected_points"] = round(
    (test_df["tx_points"] / test_df["tx_gp"]) * 82
)
test_df